In [18]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np


In [13]:
## Primeiro tabulamos os tipos de ocupações existentes na base do CNES e agrpamos


query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `basedosdados-projetos.republica.vinculo_saude_tipo_ocupacao` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"

In [14]:
df=bd.read_sql(query, billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 1377636/1377636 [01:33<00:00, 14780.96rows/s]


In [5]:
## salvando df para não precisar fazer requisição mais vezes. 
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\CNES")

In [15]:
df.to_csv('CNES_profissionais_saude_ocupacao.csv',index=False)

In [70]:
df= pd.read_csv('CNES_profissionais_saude_ocupacao.csv')

In [71]:
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.startswith('Médicos'),'Médicos',df['tipo_ocupacao'])
## Aqui eu unifico os três tipos de Médicos (clínica, cirurgia e terapeutico)

In [72]:
df = df[(df['tipo_2']=='Médicos') & (df['ano']>2017)]

In [35]:
## Chamando a quantidade de população.  

pop = pd.read_csv("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\IBGE\\Censo_previa_pop.csv", decimal=',', sep=';')

In [37]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
210230,2016,Médicos clínicos,BA,2927408,4226,Médicos
210231,2016,Médicos em especialidades cirúrgicas,BA,2924009,13,Médicos
210238,2016,Médicos clínicos,GO,5208707,2997,Médicos
210240,2016,Médicos clínicos,MA,2104701,5,Médicos
210261,2016,Médicos em especialidades cirúrgicas,RS,4314902,1242,Médicos
210271,2016,Médicos em especialidades cirúrgicas,SE,2800308,839,Médicos
210284,2016,Médicos clínicos,RR,1400100,732,Médicos
210289,2016,Médicos em especialidades cirúrgicas,RR,1400100,286,Médicos
210332,2016,Médicos em especialidades cirúrgicas,AM,1302603,1310,Médicos
210344,2016,Médicos clínicos,AM,1302603,1888,Médicos


In [73]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

In [74]:
x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

In [75]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [76]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [68]:
sum(df1[df1['ano']==2021]['POPULAÇÃO'])

207797413.0

In [67]:
sum(df1[df1['ano']==2021]['quantidade_vinculos'])/sum(df1[df1['ano']==2021]['POPULAÇÃO'])*1000

2.333277363756208

In [77]:
## validando alguns dados
df1[(df1['ano']==2021) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
14606,2021,RJ,3304110.0,Médicos,135,RJ,Porto Real,20375.0,6.625767
14666,2021,RJ,3305000.0,Médicos,231,RJ,São João da Barra,36518.0,6.325648
14594,2021,RJ,3303955.0,Médicos,136,RJ,Pinheiral,24553.0,5.539038
14634,2021,RJ,3304508.0,Médicos,50,RJ,Rio das Flores,9387.0,5.326515
14530,2021,RJ,3302601.0,Médicos,201,RJ,Mangaratiba,38953.0,5.160065
14426,2021,RJ,3300902.0,Médicos,73,RJ,Cambuci,14245.0,5.124605
14390,2021,RJ,3300233.0,Médicos,197,RJ,Armação dos Búzios,39033.0,5.047012
14738,2021,RJ,3306305.0,Médicos,1346,RJ,Volta Redonda,270543.0,4.975180
14562,2021,RJ,3303302.0,Médicos,2577,RJ,Niterói,523664.0,4.921094
14378,2021,RJ,3300159.0,Médicos,53,RJ,Aperibé,10893.0,4.865510


In [69]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [81]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [83]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_ocupacao',
       'quantidade_vinculos', 'nome_municipio', 'populacao_domiciliada',
       'taxa'],
      dtype='object')

In [79]:
## Subindo para GBQ

client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao') # verificar a categoria do dado

In [84]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',  
       'taxa']]

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22274 entries, 21386 to 22273
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    22274 non-null  int64  
 1   sigla_uf               22274 non-null  object 
 2   nome_municipio         22274 non-null  object 
 3   id_municipio           22274 non-null  float64
 4   populacao_domiciliada  22274 non-null  float64
 5   tipo_ocupacao          22274 non-null  object 
 6   quantidade_vinculos    22274 non-null  int64  
 7   taxa                   22274 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 1.5+ MB


In [86]:

schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','Float',description='Taxa de médicos por 1000 habitantes')
 ]


In [87]:
table_ref = dataset_ref.table('CNES_medicos_mil_habitantes') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=908e995a-db64-4882-ad3c-bf2bdd55fb3f>